In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spam classifier').getOrCreate()

In [4]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True,
                     sep='\t')

In [8]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','message')
data.show()

+-----+--------------------+
|class|             message|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



# Clean and Prepare the Data
** Create a new length feature: **

In [9]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['message']))
data.show()

+-----+--------------------+------+
|class|             message|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



#lets get some insights for the difference in the length for the messages of these two class messages that asre spam and ham

In [11]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



Now we get the idea about the average difference in length of the messages

# Lets do some actual text Processing for the data

**Preparing the data**

<b>Feature Transformation</b>

In [13]:
from pyspark.ml.feature import (Tokenizer, CountVectorizer, 
                                StopWordsRemover, IDF, StringIndexer)

tokenizer = Tokenizer(inputCol = 'message', outputCol = 'token')
stop_words = StopWordsRemover(inputCol='token', outputCol='stoptoken')
c_vec = CountVectorizer(inputCol='stoptoken', outputCol = 'count_vec')
idf = IDF(inputCol='count_vec', outputCol = 'tf_idf')
ham_spam_label = StringIndexer(inputCol='class', outputCol='label')

In [14]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols = ['tf_idf','length'],
    outputCol = 'features')


In [16]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

#lets get a pipeline and lined up our feature transformation steps so that they will be carried out in a sequence vis a pipeline object

In [17]:
from pyspark.ml import Pipeline

In [18]:
data_pipelines = Pipeline(stages = [ham_spam_label,tokenizer, stop_words, c_vec, idf, assembler])

In [19]:
cleandata = data_pipelines.fit(data).transform(data)

In [20]:
cleandata.printSchema()

root
 |-- class: string (nullable = true)
 |-- message: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = true)
 |-- token: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stoptoken: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- count_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)



In [22]:
mldata = cleandata.select('features','label')
mldata.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(13424,[7,11,31,6...|  0.0|
|(13424,[0,24,297,...|  0.0|
|(13424,[2,13,19,3...|  1.0|
|(13424,[0,70,80,1...|  0.0|
|(13424,[36,134,31...|  0.0|
|(13424,[10,60,139...|  1.0|
|(13424,[10,53,103...|  0.0|
|(13424,[125,184,4...|  0.0|
|(13424,[1,47,118,...|  1.0|
|(13424,[0,1,13,27...|  1.0|
|(13424,[18,43,120...|  0.0|
|(13424,[8,17,37,8...|  1.0|
|(13424,[13,30,47,...|  1.0|
|(13424,[39,96,217...|  0.0|
|(13424,[552,1697,...|  0.0|
|(13424,[30,109,11...|  1.0|
|(13424,[82,214,47...|  0.0|
|(13424,[0,2,49,13...|  0.0|
|(13424,[0,74,105,...|  0.0|
|(13424,[4,30,33,5...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [23]:
#Making train-test splits
train, test = mldata.randomSplit([0.8,0.2])

In [24]:
#lets get our model
classifier = nb.fit(train)

In [25]:
#deploy it on test data
test_results = classifier.transform(test)

In [26]:
test_results.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(13424,[0,1,2,20,...|  1.0|[-1163.0881608848...|[2.37506684030818...|       1.0|
|(13424,[0,1,4,13,...|  1.0|[-1402.6063348349...|[1.49150553040185...|       1.0|
|(13424,[0,1,7,15,...|  0.0|[-658.95819652588...|[1.0,7.3168063391...|       0.0|
|(13424,[0,1,11,32...|  0.0|[-886.31676528524...|[1.0,8.8798391632...|       0.0|
|(13424,[0,1,13,18...|  1.0|[-1170.3386704834...|[3.14961771197959...|       1.0|
|(13424,[0,1,14,31...|  0.0|[-215.27949490282...|[1.0,1.5083918451...|       0.0|
|(13424,[0,1,47,59...|  1.0|[-1376.9629711573...|[1.47918499536610...|       1.0|
|(13424,[0,1,874,1...|  0.0|[-95.036910642319...|[0.99999999373945...|       0.0|
|(13424,[0,1,3657,...|  0.0|[-128.73642119737...|[0.99996715597262...|       0.0|
|(13424,[0,2,3,4

# Evaluating our model

In [27]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)

In [32]:
bineval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
area = bineval.evaluate(test_results)
print("The area under ROC curve for Naive Bayes model for test dataset : {}".format(area))

The area under ROC curve for Naive Bayes model for test dataset : 0.940118028234205


this result is pretty good.
Now lets get some accuracy metric for this

In [34]:
acc = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = acc.evaluate(test_results)

print("Accuracy : {0:0.2f}%".format(accuracy*100))

Accuracy : 92.07%
